<a href="https://colab.research.google.com/github/DasBytes/three-stage-banglish-depression-classifier/blob/main/Banglish_Depression_classifier_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Upload Code

In [6]:
from google.colab import files
uploaded = files.upload()

Saving Banglish depression dataset.csv to Banglish depression dataset.csv


#Logistic regression

In [16]:
import pandas as pd
import numpy as np
import re
from sklearn.utils import shuffle, resample
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.preprocessing import StandardScaler
from scipy.sparse import hstack

file_name = list(uploaded.keys())[0]
df = pd.read_csv(file_name, header=None, names=["Category", "Sentence"])
df.dropna(subset=["Sentence", "Category"], inplace=True)
df["Category"] = df["Category"].str.strip()
df = shuffle(df, random_state=42)

def clean_text(text):
    if not isinstance(text, str):
        return ""
    text = text.lower()
    text = re.sub(r"http\S+|www\S+", "", text)
    text = re.sub(r"[^a-zA-Zআ-হ0-9\s]", " ", text)
    return re.sub(r"\s+", " ", text).strip()

df["Cleaned"] = df["Sentence"].apply(clean_text)

positive_words = ['valo', 'bhalo', 'happy', 'alhamdulillah', 'nice']
negative_words = ['kharap', 'na', 'tired', 'stress', 'sad', 'suicide', 'khub']

def count_words(text, word_list):
    return sum(text.count(w) for w in word_list)

df["sent_len"] = df["Cleaned"].apply(lambda x: len(x.split()))
df["pos_count"] = df["Cleaned"].apply(lambda x: count_words(x, positive_words))
df["neg_count"] = df["Cleaned"].apply(lambda x: count_words(x, negative_words))

classes = df["Category"].unique()
max_size = df["Category"].value_counts().max()

df_balanced = pd.concat([
    resample(df[df["Category"] == cls], replace=True, n_samples=max_size, random_state=42)
    for cls in classes
])

df_balanced = shuffle(df_balanced, random_state=42)

X_text = df_balanced["Cleaned"]
X_num = df_balanced[["sent_len", "pos_count", "neg_count"]].values
y = df_balanced["Category"]

scaler = StandardScaler()
X_num = scaler.fit_transform(X_num)

X_train_text, X_test_text, y_train, y_test, X_train_num, X_test_num = train_test_split(
    X_text, y, X_num, test_size=0.2, random_state=42, stratify=y
)

logistic_vectorizer = TfidfVectorizer(max_features=5000, ngram_range=(1,4))
X_train_tfidf = logistic_vectorizer.fit_transform(X_train_text)
X_test_tfidf = logistic_vectorizer.transform(X_test_text)

X_train_combined = hstack([X_train_tfidf, X_train_num])
X_test_combined = hstack([X_test_tfidf, X_test_num])

logistic_model = LogisticRegression(
    max_iter=2000,
    multi_class='multinomial',
    solver='sag',
    C=30,
    random_state=42
)
logistic_model.fit(X_train_combined, y_train)

y_pred = logistic_model.predict(X_test_combined)
print("Accuracy :", accuracy_score(y_test, y_pred))
print("Precision:", precision_score(y_test, y_pred, average="macro"))
print("Recall   :", recall_score(y_test, y_pred, average="macro"))
print("F1-score :", f1_score(y_test, y_pred, average="macro"))

def predict_live(text):
    clean = clean_text(text)
    tfidf_vec = logistic_vectorizer.transform([clean])
    sent_len = len(clean.split())
    pos_count = count_words(clean, positive_words)
    neg_count = count_words(clean, negative_words)
    num_feat = scaler.transform([[sent_len, pos_count, neg_count]])
    combined = hstack([tfidf_vec, num_feat])
    pred = model.predict(combined)[0]
    prob = np.max(model.predict_proba(combined)) * 100
    return pred, prob

while True:
    txt = input("Enter text for prediction (or type 'exit' to quit): ").strip()
    if txt.lower() == 'exit':
        break
    if txt:
        pred, conf = predict_live(txt)
        print(f"Prediction: {pred} | Confidence: {conf:.2f}%")


/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


Accuracy : 0.9450343535290443
Precision: 0.9457987405657152
Recall   : 0.9450467581047381
F1-score : 0.945268373230606
Enter text for prediction (or type 'exit' to quit): exit


#LSTM

In [9]:
!pip install emoji
!pip install gensim
!pip install tensorflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 608.4/608.4 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.9/27.9 MB 72.2 MB/s eta 0:00:00


In [12]:
import pandas as pd
import numpy as np
import re
import emoji
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Embedding, Dropout
from tensorflow.keras.utils import to_categorical
import gensim.downloader as api
import nltk
nltk.download('stopwords')

file_name = "Banglish depression dataset.csv"
df = pd.read_csv(file_name)
df.columns = ["Category", "Sentence"]
df = df.dropna(subset=['Sentence', 'Category'])

eng_stop = set(stopwords.words("english"))
bn_stop = {"ami","tumi","amra","valo","kharap","ache","achhi","kintu","na","ar","shob",
           "ekta","kore","shudhu","amar","tumar","jibone","mone","kotha","ki","kemon",
           "tome","tomar","tara","tarao","taraor","je","sei","ei","oka","ora"}
stop_words = eng_stop.union(bn_stop)

positive_words = {'moja','happy','bhalo','fun','sundor','friend','party','mojar'}
negative_words = {'dukho','kharaap','niras','lonely','stress','dukhi','depressed'}

tokenizer = RegexpTokenizer(r'\w+')

def clean_text(text):
    text = str(text).lower()
    text = emoji.replace_emoji(text, replace='')
    text = re.sub(r"http\S+|www\S+", "", text)
    text = re.sub(r"[^a-zA-Z\u0980-\u09FF ]+", " ", text)
    text = re.sub(r"\s+", " ", text).strip()
    tokens = tokenizer.tokenize(text)
    tokens = [t for t in tokens if t not in stop_words and len(t) > 1]
    return tokens

def augment_text(tokens):
    if len(tokens) > 1:
        idx = np.random.randint(0, len(tokens))
        tokens.insert(idx, tokens[idx])
    return tokens

aug_sentences = []
aug_labels = []
for sentence, label in zip(df['Sentence'], df['Category']):
    tokens = clean_text(sentence)
    aug_sentences.append(tokens)
    aug_labels.append(label)
    aug_sentences.append(augment_text(tokens.copy()))
    aug_labels.append(label)

df_aug = pd.DataFrame({"Category": aug_labels, "tokens": aug_sentences})
df_aug = shuffle(df_aug, random_state=42)

ft_model = api.load("fasttext-wiki-news-subwords-300")
embedding_dim = ft_model.vector_size

word_index = {word: idx+1 for idx, word in enumerate(ft_model.key_to_index)}
vocab_size = len(word_index) + 1

def tokens_to_sequence(tokens):
    return [word_index[t] for t in tokens if t in word_index]

df_aug['seq'] = df_aug['tokens'].apply(tokens_to_sequence)
max_len = 50
X = pad_sequences(df_aug['seq'], maxlen=max_len)

embedding_matrix = np.zeros((vocab_size, embedding_dim))
for word, idx in word_index.items():
    try:
        embedding_matrix[idx] = ft_model.get_vector(word)
    except KeyError:
        continue

encoder = LabelEncoder()
y = encoder.fit_transform(df_aug['Category'])
y_cat = to_categorical(y)
num_classes = y_cat.shape[1]

X_train, X_test, y_train, y_test = train_test_split(
    X, y_cat, test_size=0.2, random_state=42, stratify=y
)

lstm_model = Sequential()
lstm_model.add(Embedding(input_dim=vocab_size, output_dim=embedding_dim,
                    weights=[embedding_matrix], input_length=max_len, trainable=False))
lstm_model.add(LSTM(128))
lstm_model.add(Dropout(0.3))
lstm_model.add(Dense(64, activation='relu'))
lstm_model.add(Dropout(0.3))
lstm_model.add(Dense(num_classes, activation='softmax'))

lstm_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
history = lstm_model.fit(X_train, y_train, epochs=15, batch_size=32, validation_split=0.1, verbose=1)

y_pred_probs = lstm_model.predict(X_test)
y_pred_classes = np.argmax(y_pred_probs, axis=1)
y_true_classes = np.argmax(y_test, axis=1)

accuracy = accuracy_score(y_true_classes, y_pred_classes)
precision = precision_score(y_true_classes, y_pred_classes, average='weighted', zero_division=0)
recall = recall_score(y_true_classes, y_pred_classes, average='weighted', zero_division=0)
f1 = f1_score(y_true_classes, y_pred_classes, average='weighted', zero_division=0)

print("✨ lstm_model Evaluation Results ✨")
print(f"Accuracy:  {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall:    {recall:.4f}")
print(f"F1 Score:  {f1:.4f}")

def predict_sentence_with_confidence(sentence):
    tokens = clean_text(sentence)
    if len(set(tokens) & positive_words) > 0:
        return "Non-depression", 1.0
    seq = tokens_to_sequence(tokens)
    padded = pad_sequences([seq], maxlen=max_len)
    pred = lstm_model.predict(padded)
    class_idx = np.argmax(pred)
    class_label = encoder.inverse_transform([class_idx])[0]
    confidence = pred[0][class_idx]
    return class_label, confidence

while True:
    sentence = input("Enter a Banglish sentence (or type 'exit' to quit): ")
    if sentence.lower() == 'exit':
        break
    prediction, conf = predict_sentence_with_confidence(sentence)
    print(f"Prediction: {prediction} | Confidence: {conf:.2f}")


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


[==================================================] 100.0% 958.5/958.4MB downloaded


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Epoch 1/15
271/271 ━━━━━━━━━━━━━━━━━━━━ 29s 95ms/step - accuracy: 0.5936 - loss: 0.8134 - val_accuracy: 0.7825 - val_loss: 0.4841
Epoch 2/15
271/271 ━━━━━━━━━━━━━━━━━━━━ 39s 88ms/step - accuracy: 0.7926 - loss: 0.5079 - val_accuracy: 0.8054 - val_loss: 0.4345
Epoch 3/15
271/271 ━━━━━━━━━━━━━━━━━━━━ 27s 101ms/step - accuracy: 0.8157 - loss: 0.4431 - val_accuracy: 0.8345 - val_loss: 0.4204
Epoch 4/15
271/271 ━━━━━━━━━━━━━━━━━━━━ 25s 91ms/step - accuracy: 0.8273 - loss: 0.4184 - val_accuracy: 0.8398 - val_loss: 0.3877
Epoch 5/15
271/271 ━━━━━━━━━━━━━━━━━━━━ 25s 92ms/step - accuracy: 0.8427 - loss: 0.3837 - val_accuracy: 0.8595 - val_loss: 0.3458
Epoch 6/15
271/271 ━━━━━━━━━━━━━━━━━━━━ 25s 91ms/step - accuracy: 0.8585 - loss: 0.3418 - val_accuracy: 0.8595 - val_loss: 0.3391
Epoch 7/15
271/271 ━━━━━━━━━━━━━━━━━━━━ 41s 90ms/step - accuracy: 0.8662 - loss: 0.3284 - val_accuracy: 0.8522 - val_loss: 0.3499
Epoch 8/15
271/271 ━━━━━━━━━━━━━━━━━━━━ 43s 99ms/step - accuracy: 0.8714 - loss: 0.3136 -

#ANN MLP

In [13]:
import pandas as pd
import numpy as np
import re
import emoji
import gensim.downloader as api
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
import nltk
from nltk.corpus import stopwords

nltk.download('stopwords')

df = pd.read_csv("Banglish depression dataset.csv")
df.dropna(subset=['Sentence', 'Category'], inplace=True)

stopwords_eng = set(stopwords.words('english'))
stopwords_bangla = {'ami','tumi','shei','amra','eto','kemon','achho','aschi','na'}
all_stopwords = stopwords_eng.union(stopwords_bangla)

def clean_text(text):
    if not isinstance(text, str):
        return ""
    text = text.lower()
    text = re.sub(r'http\S+|www\S+|https\S+', '', text)
    text = re.sub(r'\S+@\S+', '', text)
    text = re.sub(r'\d+', '', text)
    text = emoji.demojize(text, delimiters=(" ", " "))
    text = re.sub(r'(.)\1{2,}', r'\1\1', text)
    tokens = text.split()
    tokens = [w for w in tokens if w not in all_stopwords]
    return tokens

df['Tokens'] = df['Sentence'].apply(clean_text)

le = LabelEncoder()
df['Label'] = le.fit_transform(df['Category'])
num_classes = len(le.classes_)
y = to_categorical(df['Label'], num_classes=num_classes)

ft_model = api.load('fasttext-wiki-news-subwords-300')
embedding_dim = ft_model.vector_size

def sentence_to_vec(tokens, model, dim):
    vecs = []
    for word in tokens:
        if word in model:
            vecs.append(model[word])
    if len(vecs) > 0:
        return np.mean(vecs, axis=0)
    else:
        return np.zeros(dim)

X = np.array([sentence_to_vec(tokens, ft_model, embedding_dim) for tokens in df['Tokens']])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

model = Sequential()
model.add(Dense(128, input_dim=embedding_dim, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(num_classes, activation='softmax'))

model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

model.fit(X_train, y_train, validation_split=0.1, epochs=30, batch_size=32, verbose=2)


y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true_classes = np.argmax(y_test, axis=1)

accuracy = accuracy_score(y_true_classes, y_pred_classes)
precision = precision_score(y_true_classes, y_pred_classes, average='weighted', zero_division=0)
recall = recall_score(y_true_classes, y_pred_classes, average='weighted', zero_division=0)
f1 = f1_score(y_true_classes, y_pred_classes, average='weighted', zero_division=0)

print("\n✨ Model Evaluation Results ✨")
print(f"Accuracy:  {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall:    {recall:.4f}")
print(f"F1 Score:  {f1:.4f}")

def predict_depression(text):
    tokens = clean_text(text)
    vec = sentence_to_vec(tokens, ft_model, embedding_dim).reshape(1, -1)
    pred = model.predict(vec)
    pred_class = np.argmax(pred, axis=1)[0]
    confidence = np.max(pred)
    return le.inverse_transform([pred_class])[0], confidence

while True:
    sentence = input("\nEnter text (or 'exit'): ")
    if sentence.lower() == 'exit':
        break
    category, conf = predict_depression(sentence)
    print(f"Predicted Category: {category} | Confidence: {conf:.2f}")


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Epoch 1/30


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


136/136 - 2s - 15ms/step - accuracy: 0.5869 - loss: 0.8556 - val_accuracy: 0.7131 - val_loss: 0.6974
Epoch 2/30
136/136 - 1s - 8ms/step - accuracy: 0.6906 - loss: 0.6538 - val_accuracy: 0.7484 - val_loss: 0.6025
Epoch 3/30
136/136 - 1s - 5ms/step - accuracy: 0.7489 - loss: 0.5721 - val_accuracy: 0.7651 - val_loss: 0.5502
Epoch 4/30
136/136 - 1s - 5ms/step - accuracy: 0.7838 - loss: 0.5176 - val_accuracy: 0.7380 - val_loss: 0.5808
Epoch 5/30
136/136 - 1s - 8ms/step - accuracy: 0.7887 - loss: 0.4927 - val_accuracy: 0.7630 - val_loss: 0.5392
Epoch 6/30
136/136 - 0s - 4ms/step - accuracy: 0.7991 - loss: 0.4857 - val_accuracy: 0.7900 - val_loss: 0.5064
Epoch 7/30
136/136 - 0s - 3ms/step - accuracy: 0.8079 - loss: 0.4631 - val_accuracy: 0.7879 - val_loss: 0.5048
Epoch 8/30
136/136 - 0s - 3ms/step - accuracy: 0.8144 - loss: 0.4493 - val_accuracy: 0.7879 - val_loss: 0.5012
Epoch 9/30
136/136 - 0s - 3ms/step - accuracy: 0.8174 - loss: 0.4393 - val_accuracy: 0.7734 - val_loss: 0.5073
Epoch 10/30

#Random forest


In [10]:
import pandas as pd
import numpy as np
import re
import emoji
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.utils import shuffle

file_path = 'Banglish depression dataset.csv'
df = pd.read_csv(file_path)
df.columns = ["Category", "Sentence"]
df = df.dropna(subset=['Sentence', 'Category'])

eng_stop = {"i","me","my","myself","we","our","ours","ourselves","you","your","yours","yourself","yourselves","he","him","his","himself","she","her","hers","herself","it","its","itself","they","them","their","theirs","themselves","what","which","who","whom","this","that","these","those","am","is","are","was","were","be","been","being","have","has","had","having","do","does","did","doing","a","an","the","and","but","if","or","because","as","until","while","of","at","by","for","with","about","against","between","into","through","during","before","after","above","below","to","from","up","down","in","out","on","off","over","under","again","further","then","once","here","there","when","where","why","how","all","any","both","each","few","more","most","other","some","such","no","nor","not","only","own","same","so","than","too","very","s","t","can","will","just","don","should","now"}
bn_stop = {"ami","tumi","amra","ache","achhi","kintu","na","ar","shob","ekta","kore","shudhu","amar","tumar","mone","kotha","ki","kemon","tome","tomar","tara","tarao","taraor","je","sei","ei","oka","ora"}
stop_words = eng_stop.union(bn_stop)

def preprocess_text(text):
    text = str(text).lower()
    text = emoji.demojize(text, delimiters=(" "," "))
    text = re.sub(r"http\S+|www\S+|https\S+", "", text)
    text = re.sub(r"\S+@\S+", "", text)
    text = re.sub(r"\d+", "", text)
    text = re.sub(r"[^a-zA-Z\u0980-\u09FF ]+", " ", text)
    text = re.sub(r"(.)\1{2,}", r"\1\1", text)
    text = re.sub(r"\s+", " ", text).strip()
    tokens = text.split()
    tokens = [t for t in tokens if t not in stop_words and len(t) > 1]
    return " ".join(tokens)

df['Cleaned_Sentence'] = df['Sentence'].apply(preprocess_text)

positive_words = {'moja','happy','bhalo','fun','sundor','friend','party','mojar'}
negative_words = {'dukho','kharaap','niras','lonely','stress','dukhi','depressed','chinta','udasin'}

def augment_text(text):
    tokens = text.split()
    new_tokens = tokens.copy()
    if len(tokens) > 1:
        idx = np.random.randint(0, len(tokens))
        new_tokens.insert(idx, tokens[idx])
    return " ".join(new_tokens)

aug_sentences = []
aug_labels = []
for sentence, label in zip(df['Cleaned_Sentence'], df['Category']):
    aug_sentences.append(sentence)
    aug_labels.append(label)
    for _ in range(1):
        aug_sentences.append(augment_text(sentence))
        aug_labels.append(label)

df_aug = pd.DataFrame({"Category": aug_labels, "Cleaned_Sentence": aug_sentences})
df_aug = shuffle(df_aug, random_state=42)

vectorizer = TfidfVectorizer(max_features=5000, ngram_range=(1,2))
X = vectorizer.fit_transform(df_aug['Cleaned_Sentence'])
y = df_aug['Category']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

rf_model = RandomForestClassifier(n_estimators=200, random_state=42, class_weight='balanced')
rf_model.fit(X_train, y_train)

y_pred = rf_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted', zero_division=0)
recall = recall_score(y_test, y_pred, average='weighted', zero_division=0)
f1 = f1_score(y_test, y_pred, average='weighted', zero_division=0)

print(f"Accuracy:  {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall:    {recall:.4f}")
print(f"F1 Score:  {f1:.4f}")

def predict_text(text):
    cleaned = preprocess_text(text)
    vec = vectorizer.transform([cleaned])
    pred = rf_model.predict(vec)[0]
    pred_prob = max(rf_model.predict_proba(vec)[0])
    tokens = set(cleaned.split())
    if len(tokens & positive_words) > 0:
        pred = 'Non-depression'
    if len(tokens & negative_words) > 0 and pred != 'Non-depression':
        pred_prob = min(1.0, pred_prob + 0.1)
    return pred, pred_prob

while True:
    text_input = input("Enter a Banglish sentence (or type 'exit' to quit): ")
    if text_input.lower() == 'exit':
        break
    if text_input.strip() == "":
        continue
    prediction, confidence = predict_text(text_input)
    print(f"Predicted Category: {prediction}")
    print(f"Confidence Score:   {confidence:.2f}")
    print()


Accuracy:  0.9629
Precision: 0.9631
Recall:    0.9629
F1 Score:  0.9629
Enter a Banglish sentence (or type 'exit' to quit): exit


#Saving the trained model

In [1]:
import pickle


In [52]:
import pickle

metrics = {
    "Accuracy": accuracy,
    "Precision": precision,
    "Recall": recall,
    "F1 Score": f1
}

with open("banglish_model.pkl", "wb") as f:
    pickle.dump((rf_model, vectorizer, metrics), f)



#Loading the trained model

In [29]:
!pip install --upgrade streamlit blinker --ignore-installed


  Using cached streamlit-1.53.0-py3-none-any.whl.metadata (10 kB)
  Using cached blinker-1.9.0-py3-none-any.whl.metadata (1.6 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 4.8 MB/s eta 0:00:00
  Using cached watchdog-6.0.0-py3-none-manylinux2014_x86_64.whl.metadata (44 kB)
  Using cached gitpython-3.1.46-py3-none-any.whl.metadata (13 kB)
  Using cached pydeck-0.9.1-py2.py3-none-any.whl.metadata (4.1 kB)
  Using cached gitdb-4.0.12-py3-none-any.whl.metadata (1.2 kB)
  Using cached smmap-5.0.2-py3-none-any.whl.metadata (4.3 kB)
Using cached streamlit-1.53.0-py3-none-any.whl (9.1 MB)
Using cached blinker-1.9.0-py3-none-any.whl (8.5 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 795.4/795.4 kB 22.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.3/108.3 kB 8.6 MB/s eta 0:00:00
Using cached gitpython-3.1.46-py3-none-any.whl (208 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 168.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6

In [53]:
%%writefile app.py
import streamlit as st
import pickle
import numpy as np
import re
import emoji

st.set_page_config(page_title="Banglish Depression Detector", layout="wide")
st.title("🧠 Banglish Depression Detection (ML)")

# ------------------- PREPROCESS FUNCTION -------------------
def preprocess_text(text):
    eng_stop = {"i","me","my","myself","we","our","ours","ourselves","you","your","yours",
                "yourself","yourselves","he","him","his","himself","she","her","hers",
                "herself","it","its","itself","they","them","their","theirs","themselves",
                "what","which","who","whom","this","that","these","those","am","is","are",
                "was","were","be","been","being","have","has","had","having","do","does",
                "did","doing","a","an","the","and","but","if","or","because","as","until",
                "while","of","at","by","for","with","about","against","between","into",
                "through","during","before","after","above","below","to","from","up","down",
                "in","out","on","off","over","under","again","further","then","once","here",
                "there","when","where","why","how","all","any","both","each","few","more",
                "most","other","some","such","no","nor","not","only","own","same","so",
                "than","too","very","s","t","can","will","just","don","should","now"}

    bn_stop = {"ami","tumi","amra","ache","achhi","kintu","na","ar","shob","ekta",
               "kore","shudhu","amar","tumar","mone","kotha","ki","kemon",
               "tome","tomar","tara","tarao","taraor","je","sei","ei","oka","ora"}

    stop_words = eng_stop.union(bn_stop)

    text = str(text).lower()
    text = emoji.demojize(text, delimiters=(" "," "))
    text = re.sub(r"http\S+|www\S+|https\S+", "", text)
    text = re.sub(r"\S+@\S+", "", text)
    text = re.sub(r"\d+", "", text)
    text = re.sub(r"[^a-zA-Z\u0980-\u09FF ]+", " ", text)
    text = re.sub(r"(.)\1{2,}", r"\1\1", text)
    text = re.sub(r"\s+", " ", text).strip()
    tokens = [t for t in text.split() if t not in stop_words and len(t) > 1]
    return " ".join(tokens)

# ------------------- LOAD PICKLED MODEL -------------------
@st.cache_resource
def load_model():
    with open("banglish_model.pkl", "rb") as f:
        model, vectorizer, metrics = pickle.load(f)
    return model, vectorizer, metrics

model, vectorizer, metrics = load_model()

# ------------------- SIDEBAR -------------------
st.sidebar.header("Model Performance on Test Set")
for k, v in metrics.items():
    st.sidebar.write(f"**{k}:** {v:.4f}")

st.sidebar.markdown("---")
st.sidebar.info("Trained on Banglish dataset with stopwords removed, TF-IDF + Random Forest. Includes augmentation & positive/negative word tweaks.")

# ------------------- MAIN PANEL -------------------
st.subheader("Enter a Banglish sentence for prediction:")

text_input = st.text_area("")

if st.button("Predict") and text_input.strip():
    clean = preprocess_text(text_input)
    vec = vectorizer.transform([clean])
    pred = model.predict(vec)[0]
    prob = max(model.predict_proba(vec)[0])

    # Positive/Negative words adjustment
    positive_words = {'moja','happy','bhalo','fun','sundor','friend','party','mojar'}
    negative_words = {'dukho','kharaap','niras','lonely','stress','dukhi','depressed','chinta','udasin'}
    tokens = set(clean.split())
    if len(tokens & positive_words) > 0:
        pred = 'Non-depression'
    if len(tokens & negative_words) > 0 and pred != 'Non-depression':
        prob = min(1.0, prob + 0.1)

    st.success(f"**Prediction:** {pred}")
    st.info(f"**Confidence Score:** {prob:.2f}")

    # ------------------- TOP TF-IDF WORDS -------------------
    st.subheader("Top words contributing (TF-IDF):")
    feature_names = np.array(vectorizer.get_feature_names_out())
    tfidf_vec = vec.toarray()[0]
    top_idx = tfidf_vec.argsort()[-10:][::-1]
    top_words = [(feature_names[i], tfidf_vec[i]) for i in top_idx if tfidf_vec[i] > 0]
    if top_words:
        for word, score in top_words:
            st.write(f"{word}: {score:.4f}")
    else:
        st.write("No significant words found.")


Overwriting app.py


In [55]:
!streamlit run app.py --server.port 8501 --server.address 0.0.0.0 &>/content/streamlit.log &


In [47]:
!wget -q https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64
!chmod +x cloudflared-linux-amd64


In [56]:
!./cloudflared-linux-amd64 tunnel --url http://localhost:8501


2026-01-15T15:36:47Z INF Thank you for trying Cloudflare Tunnel. Doing so, without a Cloudflare account, is a quick way to experiment and try it out. However, be aware that these account-less Tunnels have no uptime guarantee, are subject to the Cloudflare Online Services Terms of Use (https://www.cloudflare.com/website-terms/), and Cloudflare reserves the right to investigate your use of Tunnels for violations of such terms. If you intend to use Tunnels in production you should use a pre-created named tunnel by following: https://developers.cloudflare.com/cloudflare-one/connections/connect-apps
2026-01-15T15:36:47Z INF Requesting new quick Tunnel on trycloudflare.com...
2026-01-15T15:36:51Z INF +--------------------------------------------------------------------------------------------+
2026-01-15T15:36:51Z INF |  Your quick Tunnel has been created! Visit it at (it may take some time to be reachable):  |
2026-01-15T15:36:51Z INF |  https://incidents-cooperative-informational-pensions.

save and download model

In [11]:
import pickle

# --- Logistic Regression ---
with open("logistic_model.pkl", "wb") as f:
    pickle.dump((model, vectorizer, scaler), f)

# --- Random Forest ---
with open("rf_model.pkl", "wb") as f:
    pickle.dump((rf_model, vectorizer, stop_words), f)


In [14]:
# --- LSTM ---
lstm_model.save("lstm_model.h5")

# --- ANN/MLP ---
mlp_model.save("mlp_model.h5")


NameError: name 'lstm_model' is not defined